In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
class Basic_ANN(nn.Module):
    def __init__(self):
        super(Basic_ANN, self).__init__()
        self.flatten = nn.Flatten()
        self.inp_layer = nn.Linear(784, 512)
        self.hd_layer1 = nn.Linear(512, 512)
        self.op_layer = nn.Linear(512, 10)
        self.relu_func = nn.ReLU()
        self.dropoput = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu_func(self.inp_layer(x))
        x = self.dropoput(x)
        x = self.relu_func(self.hd_layer1(x))
        x = self.dropoput(x)
        x = self.op_layer(x)
        return x
    
    

In [ ]:
model = Basic_ANN().to(device)
criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
train_data = datasets.MNIST(
	root='data',
	train=True,
	download=False,
	transform=transforms.ToTensor(),
	target_transform=None
)

test_data = datasets.MNIST(
	root='data',
	train=False,
	download=False,
	transform=transforms.ToTensor(),
	target_transform={lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)}
)

In [21]:
X_train, y_train = train_data.data.to(device), train_data.targets.to(device)
X_test, y_test = test_data.data.to(device), test_data.targets.to(device)

In [22]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([60000, 28, 28]) torch.Size([60000])
torch.Size([10000, 28, 28]) torch.Size([10000])


In [ ]:
# training
import time

epoch = 5
start_time = time.time()

for i in range(epoch):
    optimizer.zero_grad()
    X_train_float = X_train.float() / 255.0 # normalize
    output = model(X_train_float)
    loss = criterian(output, y_train)
    loss.backward()
    optimizer.step()
    print("Epoch: ", i, "Loss: ", loss.item())
    
end_time = time.time()
print("Training completed in", end_time - start_time, "seconds")

Epoch:  0 Loss:  2.3005847930908203
Epoch:  1 Loss:  2.2190499305725098
Epoch:  2 Loss:  2.122945547103882
Epoch:  3 Loss:  1.9924558401107788
Epoch:  4 Loss:  1.8262786865234375
Training completed in 1.006288766860962 seconds


In [26]:
model.eval()
with torch.no_grad():
	X_test_float = X_test.float() / 255.0 # normalize
	output = model(X_test_float)
	_, pred = torch.max(output, 1)
	accuracy = torch.sum(pred == y_test).item() / len(y_test)
	print("Accuracy: ", accuracy)

Accuracy:  0.8384
